In [ ]:
import numpy as np
from numpy import mean, absolute
from scipy.spatial import distance
from scipy.linalg import pinv

from sklearn.metrics import roc_curve

import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.pyplot import Rectangle


In [ ]:
###########################################################################
# 1. Anomaly detectors
###########################################################################

# The euclideanTrain and euclideanScore functions comprise the
# Euclidean anomaly detector.  During training, the detector takes a
# set of password-timing vectors (encoded as rows in a matrix) and
# calculates the mean vector.  This mean vector is returned as the
# detection model.  During scoring, the detector takes the detection
# model and a new set of password-timing vectors (also encoded as rows
# in a matrix) and calculates the squared Euclidean distance between
# the mean vector and each of the new password-timing vectors.  These
# scores are returned in a vector whose length is equal to the number
# of password-timing vectors in the scoring matrix.

def euclideanTrain(YTrain):
    dmod = {'mean': np.mean(YTrain, axis=0)}
    return dmod


def euclideanScore(dmod, YScore):
    p = len(dmod['mean'])
    n = YScore.shape[0]

    if YScore.shape[1] != p:
        raise ValueError("Training/test feature length mismatch")

    meanMatrix = np.tile(dmod['mean'], (n, 1))

    scores = np.sum(np.square(YScore - meanMatrix), axis=1)

    return scores


# The manhattanTrain and manhattanScore functions comprise the
# Manhattan anomaly detector.  During training, the detector takes a
# set of password-timing vectors (encoded as rows in a matrix) and
# calculates the mean vector.  This mean vector is returned as the
# detection model.  During scoring, the detector takes the detection
# model and a new set of password-timing vectors (also encoded as rows
# in a matrix) and calculates the Manhattan distance between the mean
# vector and each of the new password-timing vectors.  These scores
# are returned in a vector whose length is equal to the number of
# password-timing vectors in the scoring matrix.

def manhattanTrain(YTrain):
    dmod = {'mean': np.mean(YTrain, axis=0)}
    return dmod


def manhattanScore(dmod, YScore):
    p = len(dmod['mean'])
    n = YScore.shape[0]

    if YScore.shape[1] != p:
        raise ValueError("Training/test feature length mismatch")

    meanMatrix = np.tile(dmod['mean'], (n, 1))

    scores = np.sum(np.abs(YScore - meanMatrix), axis=1)

    return scores


# Scaled Manhattan distance
def mad(data, axis=None):
    return mean(absolute(data - mean(data, axis)), axis)

def manhattanScaledTrain(YTrain):
    dmod = {'mean': np.mean(YTrain, axis=0), 'mad': mad(YTrain,0)}
    return dmod


def manhattanScaledScore(dmod, YScore):
    p = len(dmod['mean'])
    n = YScore.shape[0]
    
    if YScore.shape[1] != p:
        raise ValueError("Training/test feature length mismatch")

    meanMatrix = np.tile(dmod['mean'], (n, 1))
    scores = np.sum(np.abs(YScore - meanMatrix)/dmod["mad"].T, axis=1)
    return scores


# The mahalanobisTrain and mahalanobisScore functions comprise the
# Mahalanobis anomaly detector.  During training, the detector takes a
# set of password-timing vectors (encoded as rows in a matrix) and
# calculates the mean vector and also the inverse of the covariance
# matrix.  This vector and matrix are returned as the detection model.
# During scoring, the detector takes the detection model and a new set
# of password-timing vectors (also encoded as rows in a matrix) and
# calculates the squared Mahalanobis distance between the mean vector
# and each of the new password-timing vectors.  These scores are
# returned in a vector whose length is equal to the number of
# password-timing vectors in the scoring matrix.

def mahalanobisTrain(YTrain):
    dmod = {'mean': np.mean(YTrain, axis=0), 'covInv': pinv(
        np.cov(YTrain, rowvar=False))}
    return dmod


def mahalanobisR(X, meanCol, IC):
    m = []
    for i in range(X.shape[0]):
        m.append(distance.mahalanobis(X[i], meanCol, IC) ** 2)
    return(m)


def mahalanobisScore(dmod, YScore):
    p = len(dmod['mean'])
    n = YScore.shape[0]

    if YScore.shape[1] != p:
        raise ValueError("Training/test feature length mismatch")

    scores = mahalanobisR(YScore, dmod['mean'], dmod['covInv'])

    return scores


detectorSet = {
    'Euclidean': {
        'train': euclideanTrain,
        'score': euclideanScore
    },
    'Manhattan': {
        'train': manhattanTrain,
        'score': manhattanScore
    },
    'ManhattanScaled':{
        'train': manhattanScaledTrain,
        'score': manhattanScaledScore
    },
    'Mahalanobis': {
        'train': mahalanobisTrain,
        'score': mahalanobisScore
    }
}

In [ ]:
###########################################################################
# 2. Evaluation procedures
###########################################################################

# The calculateEqualError function takes a set of user scores and
# impostor scores, makes an ROC curve using the ROCR functionality,
# and then geometrically calculates the point at which the miss and
# false-alarm (i.e., false-negative and false-positive) rates are
# equal.


def calculateEqualError(userScores, impostorScores):

    predictions = np.concatenate((userScores, impostorScores), axis=None)
    labels = [0] * len(userScores) + [1] * len(impostorScores)

    fpr, tpr, thresholds = roc_curve(labels, predictions)
    eer = fpr[np.nanargmin(np.absolute((1 - tpr) - fpr))]

    # print(eer)
    # print(thresholds)
    #dists = tpr - fpr
    # print(dists)
    #idx1 = next(i for i, dist in enumerate(dists) if dist == min(dists[dists >= 0]))
    #idx2 = next(i for i, dist in enumerate(dists) if dist == max(dists[dists < 0]))
    # print(idx1)
    #assert len([idx1]) == 1
    #assert len([idx2]) == 1
    #assert abs(idx1 - idx2) == 1

    #x = [tpr[idx1], fpr[idx1]]
    #y = [tpr[idx2], fpr[idx2]]
    #a = (x[0] - x[1]) / (y[1] - x[1] - y[0] + x[0])
    #eer = x[0] + a * (y[0] - x[0])

    return eer


def evaluateSubject(train, eval, evalSubject, detectorTrain, detectorScore):
    # Extract the training, user scoring, and impostor scoring matrices
    # for the subject. The training matrix is the first 200 password
    # repetitions for the subject, corresponding to the first 4 sessions
    # of passwords. The user scoring matrix is the last 200 password
    # repetitions, and the impostor scoring matrix is the first 5
    # repetitions from all the other subjects.

    YTrain = train.loc[(train['subject'] == evalSubject) & (
        train['sessionIndex'] <= 4), ~train.columns.isin(['subject', 'sessionIndex', 'rep'])].values

    YScore0 = eval.loc[(eval['subject'] == evalSubject) & (
        eval['sessionIndex'] > 4), ~eval.columns.isin(['subject', 'sessionIndex', 'rep'])].values

    YScore1 = train.loc[(train['subject'] != evalSubject) & (train['sessionIndex'] == 1) & (
        train['rep'] <= 5), ~train.columns.isin(['subject', 'sessionIndex', 'rep'])].values

    # Run the training and scoring
    dmod = detectorTrain(YTrain)
    userScores = detectorScore(dmod, YScore0)
    impostorScores = detectorScore(dmod, YScore1)

    eer = calculateEqualError(userScores, impostorScores)

    return eer

In [ ]:
###########################################################################
# 3. Main procedure
###########################################################################
def eval(train, eval) -> pd.DataFrame:
    # Retrieve the list of subjects
    subjects = sorted(train["subject"].unique())

    training_set = train
    eval_set = eval


    # For each of the detectors, evaluate the detector on each subject,
    # and record the equal-error rates in a dictionary.

    eers = {}

    for detectorName, detector in detectorSet.items():

        #print(f"Evaluating the {detectorName} detector")
        detectorTrain = detector['train']
        detectorScore = detector['score']

        eers[detectorName] = np.full(len(subjects), np.nan)

        for i, subject in enumerate(subjects):
            eer = evaluateSubject(training_set, eval_set, subject,
                                detectorTrain=detectorTrain,
                                detectorScore=detectorScore)

            eers[detectorName][i] = eer
            #print(f"  {i+1}/{len(subjects)}: {eer}", end='\r')

        #print(f"  average equal-error: {np.mean(eers[detectorName])}")

    #print("Tabulating results:")

    eers = pd.DataFrame(eers, index=subjects)

    res = pd.DataFrame({'err.mean': eers.mean(), 'err.std': eers.std()})
    res = res.round(decimals=3)

    return res

In [ ]:
# Original
original = pd.read_csv('./DSL-StrongPasswordData.csv')
original_out = eval(original, original)

In [ ]:
# Chromium Isolated
chromium_isolated = pd.read_csv('./chromium/password_data_chromium_isolated_warmup/full_data_set.csv')
chromium_isolated_ot = eval(original, chromium_isolated)
chromium_isolated_tt = eval(chromium_isolated, chromium_isolated)
chromium_isolated_to = eval(chromium_isolated, original)

In [ ]:
# Chromium Uniosolated
chromium_unisolated = pd.read_csv('./chromium/password_data_chromium_unisolated_warmup/full_data_set.csv')
chromium_unisolated_ot = eval(original, chromium_unisolated)
chromium_unisolated_tt = eval(chromium_unisolated, chromium_unisolated)
chromium_unisolated_to = eval(chromium_unisolated, original)

In [ ]:
# Chrome Isolated
chrome_isolated = pd.read_csv('./chrome/password_data_chrome_isolated_warmup/full_data_set.csv')
chrome_isolated_ot = eval(original, chrome_isolated)
chrome_isolated_tt = eval(chrome_isolated, chrome_isolated)
chrome_isolated_to = eval(chrome_isolated, original)

In [ ]:
# Chrome Uniosolated
chrome_unisolated = pd.read_csv('./chrome/password_data_chrome_unisolated_warmup/full_data_set.csv')
chrome_unisolated_ot = eval(original, chrome_unisolated)
chrome_unisolated_tt = eval(chrome_unisolated, chrome_unisolated)
chrome_unisolated_to = eval(chrome_unisolated, original)

In [ ]:
# Edge Isolated
edge_isolated = pd.read_csv('./edge/password_data_edge_isolated_warmup/full_data_set.csv')
edge_isolated_ot = eval(original, edge_isolated)
edge_isolated_tt = eval(edge_isolated, edge_isolated)
edge_isolated_to = eval(edge_isolated, original)

In [ ]:
# Edge Uniosolated
edge_unisolated = pd.read_csv('./edge/password_data_edge_unisolated_warmup/full_data_set.csv')
edge_unisolated_ot = eval(original, edge_isolated)
edge_unisolated_tt = eval(edge_isolated, edge_isolated)
edge_unisolated_to = eval(edge_isolated, original)

In [ ]:
# Safari Isolated
safari_isolated = pd.read_csv('./safari/password_data_safari_isolated_warmup/full_data_set.csv')
safari_isolated_ot = eval(original, safari_isolated)
safari_isolated_tt = eval(safari_isolated, safari_isolated)
safari_isolated_to = eval(safari_isolated, original)

In [ ]:
# Safari Uniosolated
safari_unisolated = pd.read_csv('./safari/password_data_safari_unisolated_warmup/full_data_set.csv')
safari_unisolated_ot = eval(original, safari_unisolated)
safari_unisolated_tt = eval(safari_unisolated, safari_unisolated)
safari_unisolated_to = eval(safari_unisolated, original)

In [ ]:
# Firefox Isolated
firefox_isolated = pd.read_csv('./firefox/password_data_firefox_isolated_warmup/full_data_set.csv')
firefox_isolated_ot = eval(original, firefox_isolated)
firefox_isolated_tt = eval(firefox_isolated, firefox_isolated)
firefox_isolated_to = eval(firefox_isolated, original)

In [ ]:
# Firefox Uniosolated
firefox_unisolated = pd.read_csv('./firefox/password_data_firefox_unisolated_warmup/full_data_set.csv')
firefox_unisolated_ot = eval(original, firefox_unisolated)
firefox_unisolated_tt = eval(firefox_unisolated, firefox_unisolated)
firefox_unisolated_to = eval(firefox_unisolated, original)

In [ ]:
# Tor Uniosolated
tor_unisolated = pd.read_csv('./tor/password_data_tor_unisolated_warmup/full_data_set.csv')
tor_unisolated_ot = eval(original, tor_unisolated)
tor_unisolated_tt = eval(tor_unisolated, tor_unisolated)
tor_unisolated_to = eval(tor_unisolated, original)

In [ ]:
# Firefox unisolated | resistFingerprinting
firefox_unisolated_resist = pd.read_csv('./firefox_resistFP/firefox_unisolated_resistFP/full_data_set.csv')
firefox_unisolated_resist_out_ot = eval(original, firefox_unisolated_resist)
firefox_unisolated_resist_out_tt = eval(firefox_unisolated_resist, firefox_unisolated_resist)
firefox_unisolated_resist_out_to = eval(firefox_unisolated_resist, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 20ms
firefox_unisolated_resist_20ms = pd.read_csv('./firefox_resistFP_20/firefox_unisolated_resistFP_20ms/full_data_set.csv')
firefox_unisolated_resist_20ms_out_ot = eval(original, firefox_unisolated_resist_20ms)
firefox_unisolated_resist_20ms_out_tt = eval(firefox_unisolated_resist_20ms, firefox_unisolated_resist_20ms)
firefox_unisolated_resist_20ms_out_to = eval(firefox_unisolated_resist_20ms, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 33ms
firefox_unisolated_resist_33ms = pd.read_csv('./firefox_resistFP_33/firefox_unisolated_resistFP_33ms/full_data_set.csv')
firefox_unisolated_resist_33ms_out_ot = eval(original, firefox_unisolated_resist_33ms)
firefox_unisolated_resist_33ms_out_tt = eval(firefox_unisolated_resist_33ms, firefox_unisolated_resist_33ms)
firefox_unisolated_resist_33ms_out_to = eval(firefox_unisolated_resist_33ms, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 40ms
firefox_unisolated_resist_40ms = pd.read_csv('./firefox_resistFP_40/firefox_unisolated_resistFP_40ms/full_data_set.csv')
firefox_unisolated_resist_40ms_out_ot = eval(original, firefox_unisolated_resist_40ms)
firefox_unisolated_resist_40ms_out_tt = eval(firefox_unisolated_resist_40ms, firefox_unisolated_resist_40ms)
firefox_unisolated_resist_40ms_out_to = eval(firefox_unisolated_resist_40ms, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 60ms
firefox_unisolated_resist_60ms = pd.read_csv('./firefox_resistFP_60/firefox_unisolated_resistFP_60ms/full_data_set.csv')
firefox_unisolated_resist_60ms_out_ot = eval(original, firefox_unisolated_resist_60ms)
firefox_unisolated_resist_60ms_out_tt = eval(firefox_unisolated_resist_60ms, firefox_unisolated_resist_60ms)
firefox_unisolated_resist_60ms_out_to = eval(firefox_unisolated_resist_60ms, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 80ms
firefox_unisolated_resist_80ms = pd.read_csv('./firefox_resistFP_80/firefox_unisolated_resistFP_80ms/full_data_set.csv')
firefox_unisolated_resist_80ms_out_ot = eval(original, firefox_unisolated_resist_80ms)
firefox_unisolated_resist_80ms_out_tt = eval(firefox_unisolated_resist_80ms, firefox_unisolated_resist_80ms)
firefox_unisolated_resist_80ms_out_to = eval(firefox_unisolated_resist_80ms, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 100ms
firefox_unisolated_resist_100ms = pd.read_csv('./firefox_resistFP_100/firefox_unisolated_resistFP_100ms/full_data_set.csv')
firefox_unisolated_resist_100ms_out_ot = eval(original, firefox_unisolated_resist_100ms)
firefox_unisolated_resist_100ms_out_tt = eval(firefox_unisolated_resist_100ms, firefox_unisolated_resist_100ms)
firefox_unisolated_resist_100ms_out_to = eval(firefox_unisolated_resist_100ms, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 120ms
firefox_unisolated_resist_120ms = pd.read_csv('./firefox_resistFP_120/firefox_unisolated_resistFP_120ms/full_data_set.csv')
firefox_unisolated_resist_120ms_out_ot = eval(original, firefox_unisolated_resist_120ms)
firefox_unisolated_resist_120ms_out_tt = eval(firefox_unisolated_resist_120ms, firefox_unisolated_resist_120ms)
firefox_unisolated_resist_120ms_out_to = eval(firefox_unisolated_resist_120ms, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 140ms
firefox_unisolated_resist_140ms = pd.read_csv('./firefox_resistFP_140/firefox_unisolated_resistFP_140ms/full_data_set.csv')
firefox_unisolated_resist_140ms_out_ot = eval(original,firefox_unisolated_resist_140ms)
firefox_unisolated_resist_140ms_out_tt = eval(firefox_unisolated_resist_140ms, firefox_unisolated_resist_140ms)
firefox_unisolated_resist_140ms_out_to = eval(firefox_unisolated_resist_140ms, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 160ms
firefox_unisolated_resist_160ms = pd.read_csv('./firefox_resistFP_160/firefox_unisolated_resistFP_160ms/full_data_set.csv')
firefox_unisolated_resist_160ms_out_ot = eval(original,firefox_unisolated_resist_160ms)
firefox_unisolated_resist_160ms_out_tt = eval(firefox_unisolated_resist_160ms, firefox_unisolated_resist_160ms)
firefox_unisolated_resist_160ms_out_to = eval(firefox_unisolated_resist_160ms, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 180ms
firefox_unisolated_resist_180ms = pd.read_csv('./firefox_resistFP_180/firefox_unisolated_resistFP_180ms/full_data_set.csv')
firefox_unisolated_resist_180ms_out_ot = eval(original, firefox_unisolated_resist_180ms)
firefox_unisolated_resist_180ms_out_tt = eval(firefox_unisolated_resist_180ms, firefox_unisolated_resist_180ms)
firefox_unisolated_resist_180ms_out_to = eval(firefox_unisolated_resist_180ms, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 200ms
firefox_unisolated_resist_200ms = pd.read_csv('./firefox_resistFP_200/firefox_unisolated_resistFP_200ms/full_data_set.csv')
firefox_unisolated_resist_200ms_out_ot = eval(original, firefox_unisolated_resist_200ms)
firefox_unisolated_resist_200ms_out_tt = eval(firefox_unisolated_resist_200ms, firefox_unisolated_resist_200ms)
firefox_unisolated_resist_200ms_out_to = eval(firefox_unisolated_resist_200ms, original)

In [ ]:
# Firefox unisolated | resistFingerprinting NJ
firefox_unisolated_resist_NJ = pd.read_csv('./firefox_resistFP_NJ/firefox_unisolated_resistFP_NJ/full_data_set.csv')
firefox_unisolated_resist_NJ_out_ot = eval(original, firefox_unisolated_resist_NJ)
firefox_unisolated_resist_NJ_out_tt = eval(firefox_unisolated_resist_NJ, firefox_unisolated_resist_NJ)
firefox_unisolated_resist_NJ_out_to = eval(firefox_unisolated_resist_NJ, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 20ms NJ
firefox_unisolated_resist_20ms_NJ = pd.read_csv('./firefox_resistFP_20_NJ/firefox_unisolated_resistFP_20ms_NJ/full_data_set.csv')
firefox_unisolated_resist_20ms_NJ_out_ot = eval(original, firefox_unisolated_resist_20ms_NJ)
firefox_unisolated_resist_20ms_NJ_out_tt = eval(firefox_unisolated_resist_20ms_NJ, firefox_unisolated_resist_20ms_NJ)
firefox_unisolated_resist_20ms_NJ_out_to = eval(firefox_unisolated_resist_20ms_NJ, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 33ms NJ
firefox_unisolated_resist_33ms_NJ = pd.read_csv('./firefox_resistFP_33_NJ/firefox_unisolated_resistFP_33ms_NJ/full_data_set.csv')
firefox_unisolated_resist_33ms_NJ_out_ot = eval(original, firefox_unisolated_resist_33ms_NJ)
firefox_unisolated_resist_33ms_NJ_out_tt = eval(firefox_unisolated_resist_33ms_NJ, firefox_unisolated_resist_33ms_NJ)
firefox_unisolated_resist_33ms_NJ_out_to = eval(firefox_unisolated_resist_33ms_NJ, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 40ms NJ
firefox_unisolated_resist_40ms_NJ = pd.read_csv('./firefox_resistFP_40_NJ/firefox_unisolated_resistFP_40ms_NJ/full_data_set.csv')
firefox_unisolated_resist_40ms_NJ_out_ot = eval(original, firefox_unisolated_resist_40ms_NJ)
firefox_unisolated_resist_40ms_NJ_out_tt = eval(firefox_unisolated_resist_40ms_NJ, firefox_unisolated_resist_40ms_NJ)
firefox_unisolated_resist_40ms_NJ_out_to = eval(firefox_unisolated_resist_40ms_NJ, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 60ms NJ
firefox_unisolated_resist_60ms_NJ = pd.read_csv('./firefox_resistFP_60_NJ/firefox_unisolated_resistFP_60ms_NJ/full_data_set.csv')
firefox_unisolated_resist_60ms_NJ_out_ot = eval(original, firefox_unisolated_resist_60ms_NJ)
firefox_unisolated_resist_60ms_NJ_out_tt = eval(firefox_unisolated_resist_60ms_NJ, firefox_unisolated_resist_60ms_NJ)
firefox_unisolated_resist_60ms_NJ_out_to = eval(firefox_unisolated_resist_60ms_NJ, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 80ms NJ
firefox_unisolated_resist_80ms_NJ = pd.read_csv('./firefox_resistFP_80_NJ/firefox_unisolated_resistFP_80ms_NJ/full_data_set.csv')
firefox_unisolated_resist_80ms_NJ_out_ot = eval(original, firefox_unisolated_resist_80ms_NJ)
firefox_unisolated_resist_80ms_NJ_out_tt = eval(firefox_unisolated_resist_80ms_NJ, firefox_unisolated_resist_80ms_NJ)
firefox_unisolated_resist_80ms_NJ_out_to = eval(firefox_unisolated_resist_80ms_NJ, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 100ms NJ
firefox_unisolated_resist_100ms_NJ = pd.read_csv('./firefox_resistFP_100_NJ/firefox_unisolated_resistFP_100ms_NJ/full_data_set.csv')
firefox_unisolated_resist_100ms_NJ_out_ot = eval(original, firefox_unisolated_resist_100ms_NJ)
firefox_unisolated_resist_100ms_NJ_out_tt = eval(firefox_unisolated_resist_100ms_NJ, firefox_unisolated_resist_100ms_NJ)
firefox_unisolated_resist_100ms_NJ_out_to = eval(firefox_unisolated_resist_100ms_NJ, original)

In [ ]:
# Firefox unisolated| resistFingerprinting 120ms NJ
firefox_unisolated_resist_120ms_NJ = pd.read_csv('./firefox_resistFP_120_NJ/firefox_unisolated_resistFP_120ms_NJ/full_data_set.csv')
firefox_unisolated_resist_120ms_NJ_out_ot = eval(original, firefox_unisolated_resist_120ms_NJ)
firefox_unisolated_resist_120ms_NJ_out_tt = eval(firefox_unisolated_resist_120ms_NJ, firefox_unisolated_resist_120ms_NJ)
firefox_unisolated_resist_120ms_NJ_out_to = eval(firefox_unisolated_resist_120ms_NJ, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 140ms NJ
firefox_unisolated_resist_140ms_NJ = pd.read_csv('./firefox_resistFP_140_NJ/firefox_unisolated_resistFP_140ms_NJ/full_data_set.csv')
firefox_unisolated_resist_140ms_NJ_out_ot = eval(original, firefox_unisolated_resist_140ms_NJ)
firefox_unisolated_resist_140ms_NJ_out_tt = eval(firefox_unisolated_resist_140ms_NJ, firefox_unisolated_resist_140ms_NJ)
firefox_unisolated_resist_140ms_NJ_out_to = eval(firefox_unisolated_resist_140ms_NJ, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 160ms NJ
firefox_unisolated_resist_160ms_NJ = pd.read_csv('./firefox_resistFP_160_NJ/firefox_unisolated_resistFP_160ms_NJ/full_data_set.csv')
firefox_unisolated_resist_160ms_NJ_out_ot = eval(original, firefox_unisolated_resist_160ms_NJ)
firefox_unisolated_resist_160ms_NJ_out_tt = eval(firefox_unisolated_resist_160ms_NJ, firefox_unisolated_resist_160ms_NJ)
firefox_unisolated_resist_160ms_NJ_out_to = eval(firefox_unisolated_resist_160ms_NJ, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 180ms NJ
firefox_unisolated_resist_180ms_NJ = pd.read_csv('./firefox_resistFP_180_NJ/firefox_unisolated_resistFP_180ms_NJ/full_data_set.csv')
firefox_unisolated_resist_180ms_NJ_out_ot = eval(original, firefox_unisolated_resist_180ms_NJ)
firefox_unisolated_resist_180ms_NJ_out_tt = eval(firefox_unisolated_resist_180ms_NJ, firefox_unisolated_resist_180ms_NJ)
firefox_unisolated_resist_180ms_NJ_out_to = eval(firefox_unisolated_resist_180ms_NJ, original)

In [ ]:
# Firefox unisolated | resistFingerprinting 200ms NJ
firefox_unisolated_resist_200ms_NJ = pd.read_csv('./firefox_resistFP_200_NJ/firefox_unisolated_resistFP_200ms_NJ/full_data_set.csv')
firefox_unisolated_resist_200ms_NJ_out_ot = eval(original, firefox_unisolated_resist_200ms_NJ)
firefox_unisolated_resist_200ms_NJ_out_tt = eval(firefox_unisolated_resist_200ms_NJ, firefox_unisolated_resist_200ms_NJ)
firefox_unisolated_resist_200ms_NJ_out_to = eval(firefox_unisolated_resist_200ms_NJ, original)

In [ ]:
# write data | tt

df_tt = {
    "browser": [
        "Original",
        "Chromium Isolated", "Chromium Unisolated",
        "Chrome Isolated", "Chrome Unisolated",
        "Edge Isolated", "Edge Unisolated",
        "Firefox Isolated", "Firefox Unisolated",
        "Safari Isolated", "Safari Unisolated",
        "Tor Isolated",

        "Firefox RF", "Firefox RF 20ms",
        "Firefox RF 33ms", "Firefox RF 40ms",
        "Firefox RF 60ms", "Firefox RF 80ms",
        "Firefox RF 100ms", "Firefox RF 120ms",
        "Firefox RF 140ms", "Firefox RF 160ms",
        "Firefox RF 180ms", "Firefox RF 200ms",

        "Firefox RF NJ", "Firefox RF NJ 20ms",
        "Firefox RF NJ 33ms", "Firefox RF NJ 40ms",
        "Firefox RF NJ 60ms", "Firefox RF NJ 80ms",
        "Firefox RF NJ 100ms", "Firefox RF NJ 120ms",
        "Firefox RF NJ 140ms", "Firefox RF NJ 160ms",
        "Firefox RF NJ 180ms", "Firefox RF NJ 200ms",
    ],
    "Euclidean(mean)": [
        original_out.loc["Euclidean"]["err.mean"],
        chromium_isolated_tt.loc["Euclidean"]["err.mean"], chromium_unisolated_tt.loc["Euclidean"]["err.mean"],
        chrome_isolated_tt.loc["Euclidean"]["err.mean"], chrome_unisolated_tt.loc["Euclidean"]["err.mean"],
        edge_isolated_tt.loc["Euclidean"]["err.mean"], edge_unisolated_tt.loc["Euclidean"]["err.mean"],
        firefox_isolated_tt.loc["Euclidean"]["err.mean"], firefox_unisolated_tt.loc["Euclidean"]["err.mean"],
        safari_isolated_tt.loc["Euclidean"]["err.mean"], safari_unisolated_tt.loc["Euclidean"]["err.mean"],
        tor_unisolated_tt.loc["Euclidean"]["err.mean"],

        firefox_unisolated_resist_out_tt.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_20ms_out_tt.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_33ms_out_tt.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_40ms_out_tt.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_60ms_out_tt.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_80ms_out_tt.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_100ms_out_tt.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_120ms_out_tt.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_140ms_out_tt.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_160ms_out_tt.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_180ms_out_tt.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_200ms_out_tt.loc["Euclidean"]["err.mean"],

        firefox_unisolated_resist_NJ_out_tt.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_20ms_NJ_out_tt.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_33ms_NJ_out_tt.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_40ms_NJ_out_tt.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_60ms_NJ_out_tt.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_80ms_NJ_out_tt.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_100ms_NJ_out_tt.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_120ms_NJ_out_tt.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_140ms_NJ_out_tt.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_160ms_NJ_out_tt.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_180ms_NJ_out_tt.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_200ms_NJ_out_tt.loc["Euclidean"]["err.mean"],
    ],
    "Euclidean(std)": [
        original_out.loc["Euclidean"]["err.std"],
        chromium_isolated_tt.loc["Euclidean"]["err.std"], chromium_unisolated_tt.loc["Euclidean"]["err.std"],
        chrome_isolated_tt.loc["Euclidean"]["err.std"], chrome_unisolated_tt.loc["Euclidean"]["err.std"],
        edge_isolated_tt.loc["Euclidean"]["err.std"], edge_unisolated_tt.loc["Euclidean"]["err.std"],
        firefox_isolated_tt.loc["Euclidean"]["err.std"], firefox_unisolated_tt.loc["Euclidean"]["err.std"],
        safari_isolated_tt.loc["Euclidean"]["err.std"], safari_unisolated_tt.loc["Euclidean"]["err.std"],
        tor_unisolated_tt.loc["Euclidean"]["err.std"],

        firefox_unisolated_resist_out_tt.loc["Euclidean"]["err.std"], firefox_unisolated_resist_20ms_out_tt.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_33ms_out_tt.loc["Euclidean"]["err.std"], firefox_unisolated_resist_40ms_out_tt.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_60ms_out_tt.loc["Euclidean"]["err.std"], firefox_unisolated_resist_80ms_out_tt.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_100ms_out_tt.loc["Euclidean"]["err.std"], firefox_unisolated_resist_120ms_out_tt.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_140ms_out_tt.loc["Euclidean"]["err.std"], firefox_unisolated_resist_160ms_out_tt.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_180ms_out_tt.loc["Euclidean"]["err.std"], firefox_unisolated_resist_200ms_out_tt.loc["Euclidean"]["err.std"],

        firefox_unisolated_resist_NJ_out_tt.loc["Euclidean"]["err.std"], firefox_unisolated_resist_20ms_NJ_out_tt.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_33ms_NJ_out_tt.loc["Euclidean"]["err.std"], firefox_unisolated_resist_40ms_NJ_out_tt.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_60ms_NJ_out_tt.loc["Euclidean"]["err.std"], firefox_unisolated_resist_80ms_NJ_out_tt.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_100ms_NJ_out_tt.loc["Euclidean"]["err.std"], firefox_unisolated_resist_120ms_NJ_out_tt.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_140ms_NJ_out_tt.loc["Euclidean"]["err.std"], firefox_unisolated_resist_160ms_NJ_out_tt.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_180ms_NJ_out_tt.loc["Euclidean"]["err.std"], firefox_unisolated_resist_200ms_NJ_out_tt.loc["Euclidean"]["err.std"],
    ],
    
    "Manhattan(mean)": [
        original_out.loc["Manhattan"]["err.mean"],
        chromium_isolated_tt.loc["Manhattan"]["err.mean"], chromium_unisolated_tt.loc["Manhattan"]["err.mean"],
        chrome_isolated_tt.loc["Manhattan"]["err.mean"], chrome_unisolated_tt.loc["Manhattan"]["err.mean"],
        edge_isolated_tt.loc["Manhattan"]["err.mean"], edge_unisolated_tt.loc["Manhattan"]["err.mean"],
        firefox_isolated_tt.loc["Manhattan"]["err.mean"], firefox_unisolated_tt.loc["Manhattan"]["err.mean"],
        safari_isolated_tt.loc["Manhattan"]["err.mean"], safari_unisolated_tt.loc["Manhattan"]["err.mean"],
        tor_unisolated_tt.loc["Manhattan"]["err.mean"],

        firefox_unisolated_resist_out_tt.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_20ms_out_tt.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_33ms_out_tt.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_40ms_out_tt.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_60ms_out_tt.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_80ms_out_tt.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_100ms_out_tt.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_120ms_out_tt.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_140ms_out_tt.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_160ms_out_tt.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_180ms_out_tt.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_200ms_out_tt.loc["Manhattan"]["err.mean"],

        firefox_unisolated_resist_NJ_out_tt.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_20ms_NJ_out_tt.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_33ms_NJ_out_tt.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_40ms_NJ_out_tt.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_60ms_NJ_out_tt.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_80ms_NJ_out_tt.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_100ms_NJ_out_tt.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_120ms_NJ_out_tt.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_140ms_NJ_out_tt.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_160ms_NJ_out_tt.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_180ms_NJ_out_tt.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_200ms_NJ_out_tt.loc["Manhattan"]["err.mean"],
    ],
    "Manhattan(std)": [
        original_out.loc["Manhattan"]["err.std"],
        chromium_isolated_tt.loc["Manhattan"]["err.std"], chromium_unisolated_tt.loc["Manhattan"]["err.std"],
        chrome_isolated_tt.loc["Manhattan"]["err.std"], chrome_unisolated_tt.loc["Manhattan"]["err.std"],
        edge_isolated_tt.loc["Manhattan"]["err.std"], edge_unisolated_tt.loc["Manhattan"]["err.std"],
        firefox_isolated_tt.loc["Manhattan"]["err.std"], firefox_unisolated_tt.loc["Manhattan"]["err.std"],
        safari_isolated_tt.loc["Manhattan"]["err.std"], safari_unisolated_tt.loc["Manhattan"]["err.std"],
        tor_unisolated_tt.loc["Manhattan"]["err.std"],

        firefox_unisolated_resist_out_tt.loc["Manhattan"]["err.std"], firefox_unisolated_resist_20ms_out_tt.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_33ms_out_tt.loc["Manhattan"]["err.std"], firefox_unisolated_resist_40ms_out_tt.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_60ms_out_tt.loc["Manhattan"]["err.std"], firefox_unisolated_resist_80ms_out_tt.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_100ms_out_tt.loc["Manhattan"]["err.std"], firefox_unisolated_resist_120ms_out_tt.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_140ms_out_tt.loc["Manhattan"]["err.std"], firefox_unisolated_resist_160ms_out_tt.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_180ms_out_tt.loc["Manhattan"]["err.std"], firefox_unisolated_resist_200ms_out_tt.loc["Manhattan"]["err.std"],

        firefox_unisolated_resist_NJ_out_tt.loc["Manhattan"]["err.std"], firefox_unisolated_resist_20ms_NJ_out_tt.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_33ms_NJ_out_tt.loc["Manhattan"]["err.std"], firefox_unisolated_resist_40ms_NJ_out_tt.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_60ms_NJ_out_tt.loc["Manhattan"]["err.std"], firefox_unisolated_resist_80ms_NJ_out_tt.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_100ms_NJ_out_tt.loc["Manhattan"]["err.std"], firefox_unisolated_resist_120ms_NJ_out_tt.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_140ms_NJ_out_tt.loc["Manhattan"]["err.std"], firefox_unisolated_resist_160ms_NJ_out_tt.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_180ms_NJ_out_tt.loc["Manhattan"]["err.std"], firefox_unisolated_resist_200ms_NJ_out_tt.loc["Manhattan"]["err.std"],
    ],
    
    "ManhattanScaled(mean)": [
        original_out.loc["ManhattanScaled"]["err.mean"],
        chromium_isolated_tt.loc["ManhattanScaled"]["err.mean"], chromium_unisolated_tt.loc["ManhattanScaled"]["err.mean"],
        chrome_isolated_tt.loc["ManhattanScaled"]["err.mean"], chrome_unisolated_tt.loc["ManhattanScaled"]["err.mean"],
        edge_isolated_tt.loc["ManhattanScaled"]["err.mean"], edge_unisolated_tt.loc["ManhattanScaled"]["err.mean"],
        firefox_isolated_tt.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_tt.loc["ManhattanScaled"]["err.mean"],
        safari_isolated_tt.loc["ManhattanScaled"]["err.mean"], safari_unisolated_tt.loc["ManhattanScaled"]["err.mean"],
        tor_unisolated_tt.loc["ManhattanScaled"]["err.mean"],

        firefox_unisolated_resist_out_tt.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_20ms_out_tt.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_33ms_out_tt.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_40ms_out_tt.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_60ms_out_tt.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_80ms_out_tt.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_100ms_out_tt.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_120ms_out_tt.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_140ms_out_tt.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_160ms_out_tt.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_180ms_out_tt.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_200ms_out_tt.loc["ManhattanScaled"]["err.mean"],

        firefox_unisolated_resist_NJ_out_tt.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_20ms_NJ_out_tt.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_33ms_NJ_out_tt.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_40ms_NJ_out_tt.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_60ms_NJ_out_tt.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_80ms_NJ_out_tt.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_100ms_NJ_out_tt.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_120ms_NJ_out_tt.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_140ms_NJ_out_tt.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_160ms_NJ_out_tt.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_180ms_NJ_out_tt.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_200ms_NJ_out_tt.loc["ManhattanScaled"]["err.mean"],
    ],
    "ManhattanScaled(std)": [
        original_out.loc["ManhattanScaled"]["err.std"],
        chromium_isolated_tt.loc["ManhattanScaled"]["err.std"], chromium_unisolated_tt.loc["ManhattanScaled"]["err.std"],
        chrome_isolated_tt.loc["ManhattanScaled"]["err.std"], chrome_unisolated_tt.loc["ManhattanScaled"]["err.std"],
        edge_isolated_tt.loc["ManhattanScaled"]["err.std"], edge_unisolated_tt.loc["ManhattanScaled"]["err.std"],
        firefox_isolated_tt.loc["ManhattanScaled"]["err.std"], firefox_unisolated_tt.loc["ManhattanScaled"]["err.std"],
        safari_isolated_tt.loc["ManhattanScaled"]["err.std"], safari_unisolated_tt.loc["ManhattanScaled"]["err.std"],
        tor_unisolated_tt.loc["ManhattanScaled"]["err.std"],

        firefox_unisolated_resist_out_tt.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_20ms_out_tt.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_33ms_out_tt.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_40ms_out_tt.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_60ms_out_tt.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_80ms_out_tt.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_100ms_out_tt.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_120ms_out_tt.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_140ms_out_tt.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_160ms_out_tt.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_180ms_out_tt.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_200ms_out_tt.loc["ManhattanScaled"]["err.std"],

        firefox_unisolated_resist_NJ_out_tt.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_20ms_NJ_out_tt.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_33ms_NJ_out_tt.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_40ms_NJ_out_tt.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_60ms_NJ_out_tt.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_80ms_NJ_out_tt.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_100ms_NJ_out_tt.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_120ms_NJ_out_tt.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_140ms_NJ_out_tt.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_160ms_NJ_out_tt.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_180ms_NJ_out_tt.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_200ms_NJ_out_tt.loc["ManhattanScaled"]["err.std"],
    ],
    
    "Mahalanobis(mean)": [
        original_out.loc["Mahalanobis"]["err.mean"],
        chromium_isolated_tt.loc["Mahalanobis"]["err.mean"], chromium_unisolated_tt.loc["Mahalanobis"]["err.mean"],
        chrome_isolated_tt.loc["Mahalanobis"]["err.mean"], chrome_unisolated_tt.loc["Mahalanobis"]["err.mean"],
        edge_isolated_tt.loc["Mahalanobis"]["err.mean"], edge_unisolated_tt.loc["Mahalanobis"]["err.mean"],
        firefox_isolated_tt.loc["Mahalanobis"]["err.mean"], firefox_unisolated_tt.loc["Mahalanobis"]["err.mean"],
        safari_isolated_tt.loc["Mahalanobis"]["err.mean"], safari_unisolated_tt.loc["Mahalanobis"]["err.mean"],
        tor_unisolated_tt.loc["Mahalanobis"]["err.mean"],

        firefox_unisolated_resist_out_tt.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_20ms_out_tt.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_33ms_out_tt.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_40ms_out_tt.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_60ms_out_tt.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_80ms_out_tt.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_100ms_out_tt.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_120ms_out_tt.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_140ms_out_tt.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_160ms_out_tt.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_180ms_out_tt.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_200ms_out_tt.loc["ManhattanScaled"]["err.mean"],

        firefox_unisolated_resist_NJ_out_tt.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_20ms_NJ_out_tt.loc["Mahalanobis"]["err.mean"],
        firefox_unisolated_resist_33ms_NJ_out_tt.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_40ms_NJ_out_tt.loc["Mahalanobis"]["err.mean"],
        firefox_unisolated_resist_60ms_NJ_out_tt.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_80ms_NJ_out_tt.loc["Mahalanobis"]["err.mean"],
        firefox_unisolated_resist_100ms_NJ_out_tt.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_120ms_NJ_out_tt.loc["Mahalanobis"]["err.mean"],
        firefox_unisolated_resist_140ms_NJ_out_tt.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_160ms_NJ_out_tt.loc["Mahalanobis"]["err.mean"],
        firefox_unisolated_resist_180ms_NJ_out_tt.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_200ms_NJ_out_tt.loc["Mahalanobis"]["err.mean"],
    ],
    "Mahalanobis(std)": [
        original_out.loc["Mahalanobis"]["err.std"],
        chromium_isolated_tt.loc["Mahalanobis"]["err.std"], chromium_unisolated_tt.loc["Mahalanobis"]["err.std"],
        chrome_isolated_tt.loc["Mahalanobis"]["err.std"], chrome_unisolated_tt.loc["Mahalanobis"]["err.std"],
        edge_isolated_tt.loc["Mahalanobis"]["err.std"], edge_unisolated_tt.loc["Mahalanobis"]["err.std"],
        firefox_isolated_tt.loc["Mahalanobis"]["err.std"], firefox_unisolated_tt.loc["Mahalanobis"]["err.std"],
        safari_isolated_tt.loc["Mahalanobis"]["err.std"], safari_unisolated_tt.loc["Mahalanobis"]["err.std"],
        tor_unisolated_tt.loc["Mahalanobis"]["err.std"],

        firefox_unisolated_resist_out_tt.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_20ms_out_tt.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_33ms_out_tt.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_40ms_out_tt.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_60ms_out_tt.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_80ms_out_tt.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_100ms_out_tt.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_120ms_out_tt.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_140ms_out_tt.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_160ms_out_tt.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_180ms_out_tt.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_200ms_out_tt.loc["Mahalanobis"]["err.std"],

        firefox_unisolated_resist_NJ_out_tt.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_20ms_NJ_out_tt.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_33ms_NJ_out_tt.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_40ms_NJ_out_tt.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_60ms_NJ_out_tt.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_80ms_NJ_out_tt.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_100ms_NJ_out_tt.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_120ms_NJ_out_tt.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_140ms_NJ_out_tt.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_160ms_NJ_out_tt.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_180ms_NJ_out_tt.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_200ms_NJ_out_tt.loc["Mahalanobis"]["err.std"],
    ],
}

df = pd.DataFrame(df_tt, copy=False)
df.to_csv("./password_browser_browser.csv", index=False)

In [ ]:
# write data | to

df_to = {
    "browser": [
        "Original",
        "Chromium Isolated", "Chromium Unisolated",
        "Chrome Isolated", "Chrome Unisolated",
        "Edge Isolated", "Edge Unisolated",
        "Firefox Isolated", "Firefox Unisolated",
        "Safari Isolated", "Safari Unisolated",
        "Tor Isolated",

        "Firefox RF", "Firefox RF 20ms",
        "Firefox RF 33ms", "Firefox RF 40ms",
        "Firefox RF 60ms", "Firefox RF 80ms",
        "Firefox RF 100ms", "Firefox RF 120ms",
        "Firefox RF 140ms", "Firefox RF 160ms",
        "Firefox RF 180ms", "Firefox RF 200ms",

        "Firefox RF NJ", "Firefox RF NJ 20ms",
        "Firefox RF NJ 33ms", "Firefox RF NJ 40ms",
        "Firefox RF NJ 60ms", "Firefox RF NJ 80ms",
        "Firefox RF NJ 100ms", "Firefox RF NJ 120ms",
        "Firefox RF NJ 140ms", "Firefox RF NJ 160ms",
        "Firefox RF NJ 180ms", "Firefox RF NJ 200ms",
    ],
    "Euclidean(mean)": [
        original_out.loc["Euclidean"]["err.mean"],
        chromium_isolated_to.loc["Euclidean"]["err.mean"], chromium_unisolated_to.loc["Euclidean"]["err.mean"],
        chrome_isolated_to.loc["Euclidean"]["err.mean"], chrome_unisolated_to.loc["Euclidean"]["err.mean"],
        edge_isolated_to.loc["Euclidean"]["err.mean"], edge_unisolated_to.loc["Euclidean"]["err.mean"],
        firefox_isolated_to.loc["Euclidean"]["err.mean"], firefox_unisolated_to.loc["Euclidean"]["err.mean"],
        safari_isolated_to.loc["Euclidean"]["err.mean"], safari_unisolated_to.loc["Euclidean"]["err.mean"],
        tor_unisolated_to.loc["Euclidean"]["err.mean"],

        firefox_unisolated_resist_out_to.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_20ms_out_to.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_33ms_out_to.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_40ms_out_to.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_60ms_out_to.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_80ms_out_to.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_100ms_out_to.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_120ms_out_to.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_140ms_out_to.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_160ms_out_to.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_180ms_out_to.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_200ms_out_to.loc["Euclidean"]["err.mean"],

        firefox_unisolated_resist_NJ_out_to.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_20ms_NJ_out_to.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_33ms_NJ_out_to.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_40ms_NJ_out_to.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_60ms_NJ_out_to.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_80ms_NJ_out_to.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_100ms_NJ_out_to.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_120ms_NJ_out_to.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_140ms_NJ_out_to.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_160ms_NJ_out_to.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_180ms_NJ_out_to.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_200ms_NJ_out_to.loc["Euclidean"]["err.mean"],
    ],
    "Euclidean(std)": [
        original_out.loc["Euclidean"]["err.std"],
        chromium_isolated_to.loc["Euclidean"]["err.std"], chromium_unisolated_to.loc["Euclidean"]["err.std"],
        chrome_isolated_to.loc["Euclidean"]["err.std"], chrome_unisolated_to.loc["Euclidean"]["err.std"],
        edge_isolated_to.loc["Euclidean"]["err.std"], edge_unisolated_to.loc["Euclidean"]["err.std"],
        firefox_isolated_to.loc["Euclidean"]["err.std"], firefox_unisolated_to.loc["Euclidean"]["err.std"],
        safari_isolated_to.loc["Euclidean"]["err.std"], safari_unisolated_to.loc["Euclidean"]["err.std"],
        tor_unisolated_to.loc["Euclidean"]["err.std"],

        firefox_unisolated_resist_out_to.loc["Euclidean"]["err.std"], firefox_unisolated_resist_20ms_out_to.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_33ms_out_to.loc["Euclidean"]["err.std"], firefox_unisolated_resist_40ms_out_to.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_60ms_out_to.loc["Euclidean"]["err.std"], firefox_unisolated_resist_80ms_out_to.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_100ms_out_to.loc["Euclidean"]["err.std"], firefox_unisolated_resist_120ms_out_to.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_140ms_out_to.loc["Euclidean"]["err.std"], firefox_unisolated_resist_160ms_out_to.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_180ms_out_to.loc["Euclidean"]["err.std"], firefox_unisolated_resist_200ms_out_to.loc["Euclidean"]["err.std"],

        firefox_unisolated_resist_NJ_out_to.loc["Euclidean"]["err.std"], firefox_unisolated_resist_20ms_NJ_out_to.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_33ms_NJ_out_to.loc["Euclidean"]["err.std"], firefox_unisolated_resist_40ms_NJ_out_to.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_60ms_NJ_out_to.loc["Euclidean"]["err.std"], firefox_unisolated_resist_80ms_NJ_out_to.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_100ms_NJ_out_to.loc["Euclidean"]["err.std"], firefox_unisolated_resist_120ms_NJ_out_to.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_140ms_NJ_out_to.loc["Euclidean"]["err.std"], firefox_unisolated_resist_160ms_NJ_out_to.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_180ms_NJ_out_to.loc["Euclidean"]["err.std"], firefox_unisolated_resist_200ms_NJ_out_to.loc["Euclidean"]["err.std"],
    ],
    
    "Manhattan(mean)": [
        original_out.loc["Manhattan"]["err.mean"],
        chromium_isolated_to.loc["Manhattan"]["err.mean"], chromium_unisolated_to.loc["Manhattan"]["err.mean"],
        chrome_isolated_to.loc["Manhattan"]["err.mean"], chrome_unisolated_to.loc["Manhattan"]["err.mean"],
        edge_isolated_to.loc["Manhattan"]["err.mean"], edge_unisolated_to.loc["Manhattan"]["err.mean"],
        firefox_isolated_to.loc["Manhattan"]["err.mean"], firefox_unisolated_to.loc["Manhattan"]["err.mean"],
        safari_isolated_to.loc["Manhattan"]["err.mean"], safari_unisolated_to.loc["Manhattan"]["err.mean"],
        tor_unisolated_to.loc["Manhattan"]["err.mean"],

        firefox_unisolated_resist_out_to.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_20ms_out_to.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_33ms_out_to.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_40ms_out_to.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_60ms_out_to.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_80ms_out_to.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_100ms_out_to.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_120ms_out_to.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_140ms_out_to.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_160ms_out_to.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_180ms_out_to.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_200ms_out_to.loc["Manhattan"]["err.mean"],

        firefox_unisolated_resist_NJ_out_to.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_20ms_NJ_out_to.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_33ms_NJ_out_to.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_40ms_NJ_out_to.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_60ms_NJ_out_to.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_80ms_NJ_out_to.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_100ms_NJ_out_to.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_120ms_NJ_out_to.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_140ms_NJ_out_to.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_160ms_NJ_out_to.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_180ms_NJ_out_to.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_200ms_NJ_out_to.loc["Manhattan"]["err.mean"],
    ],
    "Manhattan(std)": [
        original_out.loc["Manhattan"]["err.std"],
        chromium_isolated_to.loc["Manhattan"]["err.std"], chromium_unisolated_to.loc["Manhattan"]["err.std"],
        chrome_isolated_to.loc["Manhattan"]["err.std"], chrome_unisolated_to.loc["Manhattan"]["err.std"],
        edge_isolated_to.loc["Manhattan"]["err.std"], edge_unisolated_to.loc["Manhattan"]["err.std"],
        firefox_isolated_to.loc["Manhattan"]["err.std"], firefox_unisolated_to.loc["Manhattan"]["err.std"],
        safari_isolated_to.loc["Manhattan"]["err.std"], safari_unisolated_to.loc["Manhattan"]["err.std"],
        tor_unisolated_to.loc["Manhattan"]["err.std"],

        firefox_unisolated_resist_out_to.loc["Manhattan"]["err.std"], firefox_unisolated_resist_20ms_out_to.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_33ms_out_to.loc["Manhattan"]["err.std"], firefox_unisolated_resist_40ms_out_to.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_60ms_out_to.loc["Manhattan"]["err.std"], firefox_unisolated_resist_80ms_out_to.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_100ms_out_to.loc["Manhattan"]["err.std"], firefox_unisolated_resist_120ms_out_to.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_140ms_out_to.loc["Manhattan"]["err.std"], firefox_unisolated_resist_160ms_out_to.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_180ms_out_to.loc["Manhattan"]["err.std"], firefox_unisolated_resist_200ms_out_to.loc["Manhattan"]["err.std"],

        firefox_unisolated_resist_NJ_out_to.loc["Manhattan"]["err.std"], firefox_unisolated_resist_20ms_NJ_out_to.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_33ms_NJ_out_to.loc["Manhattan"]["err.std"], firefox_unisolated_resist_40ms_NJ_out_to.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_60ms_NJ_out_to.loc["Manhattan"]["err.std"], firefox_unisolated_resist_80ms_NJ_out_to.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_100ms_NJ_out_to.loc["Manhattan"]["err.std"], firefox_unisolated_resist_120ms_NJ_out_to.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_140ms_NJ_out_to.loc["Manhattan"]["err.std"], firefox_unisolated_resist_160ms_NJ_out_to.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_180ms_NJ_out_to.loc["Manhattan"]["err.std"], firefox_unisolated_resist_200ms_NJ_out_to.loc["Manhattan"]["err.std"],
    ],
    
    "ManhattanScaled(mean)": [
        original_out.loc["ManhattanScaled"]["err.mean"],
        chromium_isolated_to.loc["ManhattanScaled"]["err.mean"], chromium_unisolated_to.loc["ManhattanScaled"]["err.mean"],
        chrome_isolated_to.loc["ManhattanScaled"]["err.mean"], chrome_unisolated_to.loc["ManhattanScaled"]["err.mean"],
        edge_isolated_to.loc["ManhattanScaled"]["err.mean"], edge_unisolated_to.loc["ManhattanScaled"]["err.mean"],
        firefox_isolated_to.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_to.loc["ManhattanScaled"]["err.mean"],
        safari_isolated_to.loc["ManhattanScaled"]["err.mean"], safari_unisolated_to.loc["ManhattanScaled"]["err.mean"],
        tor_unisolated_to.loc["ManhattanScaled"]["err.mean"],

        firefox_unisolated_resist_out_to.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_20ms_out_to.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_33ms_out_to.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_40ms_out_to.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_60ms_out_to.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_80ms_out_to.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_100ms_out_to.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_120ms_out_to.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_140ms_out_to.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_160ms_out_to.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_180ms_out_to.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_200ms_out_to.loc["ManhattanScaled"]["err.mean"],

        firefox_unisolated_resist_NJ_out_to.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_20ms_NJ_out_to.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_33ms_NJ_out_to.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_40ms_NJ_out_to.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_60ms_NJ_out_to.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_80ms_NJ_out_to.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_100ms_NJ_out_to.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_120ms_NJ_out_to.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_140ms_NJ_out_to.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_160ms_NJ_out_to.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_180ms_NJ_out_to.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_200ms_NJ_out_to.loc["ManhattanScaled"]["err.mean"],
    ],
    "ManhattanScaled(std)": [
        original_out.loc["ManhattanScaled"]["err.std"],
        chromium_isolated_to.loc["ManhattanScaled"]["err.std"], chromium_unisolated_to.loc["ManhattanScaled"]["err.std"],
        chrome_isolated_to.loc["ManhattanScaled"]["err.std"], chrome_unisolated_to.loc["ManhattanScaled"]["err.std"],
        edge_isolated_to.loc["ManhattanScaled"]["err.std"], edge_unisolated_to.loc["ManhattanScaled"]["err.std"],
        firefox_isolated_to.loc["ManhattanScaled"]["err.std"], firefox_unisolated_to.loc["ManhattanScaled"]["err.std"],
        safari_isolated_to.loc["ManhattanScaled"]["err.std"], safari_unisolated_to.loc["ManhattanScaled"]["err.std"],
        tor_unisolated_to.loc["ManhattanScaled"]["err.std"],

        firefox_unisolated_resist_out_to.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_20ms_out_to.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_33ms_out_to.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_40ms_out_to.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_60ms_out_to.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_80ms_out_to.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_100ms_out_to.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_120ms_out_to.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_140ms_out_to.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_160ms_out_to.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_180ms_out_to.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_200ms_out_to.loc["ManhattanScaled"]["err.std"],

        firefox_unisolated_resist_NJ_out_to.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_20ms_NJ_out_to.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_33ms_NJ_out_to.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_40ms_NJ_out_to.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_60ms_NJ_out_to.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_80ms_NJ_out_to.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_100ms_NJ_out_to.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_120ms_NJ_out_to.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_140ms_NJ_out_to.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_160ms_NJ_out_to.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_180ms_NJ_out_to.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_200ms_NJ_out_to.loc["ManhattanScaled"]["err.std"],
    ],
    
    "Mahalanobis(mean)": [
        original_out.loc["Mahalanobis"]["err.mean"],
        chromium_isolated_to.loc["Mahalanobis"]["err.mean"], chromium_unisolated_to.loc["Mahalanobis"]["err.mean"],
        chrome_isolated_to.loc["Mahalanobis"]["err.mean"], chrome_unisolated_to.loc["Mahalanobis"]["err.mean"],
        edge_isolated_to.loc["Mahalanobis"]["err.mean"], edge_unisolated_to.loc["Mahalanobis"]["err.mean"],
        firefox_isolated_to.loc["Mahalanobis"]["err.mean"], firefox_unisolated_to.loc["Mahalanobis"]["err.mean"],
        safari_isolated_to.loc["Mahalanobis"]["err.mean"], safari_unisolated_to.loc["Mahalanobis"]["err.mean"],
        tor_unisolated_to.loc["Mahalanobis"]["err.mean"],

        firefox_unisolated_resist_out_to.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_20ms_out_to.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_33ms_out_to.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_40ms_out_to.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_60ms_out_to.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_80ms_out_to.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_100ms_out_to.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_120ms_out_to.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_140ms_out_to.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_160ms_out_to.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_180ms_out_to.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_200ms_out_to.loc["ManhattanScaled"]["err.mean"],

        firefox_unisolated_resist_NJ_out_to.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_20ms_NJ_out_to.loc["Mahalanobis"]["err.mean"],
        firefox_unisolated_resist_33ms_NJ_out_to.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_40ms_NJ_out_to.loc["Mahalanobis"]["err.mean"],
        firefox_unisolated_resist_60ms_NJ_out_to.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_80ms_NJ_out_to.loc["Mahalanobis"]["err.mean"],
        firefox_unisolated_resist_100ms_NJ_out_to.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_120ms_NJ_out_to.loc["Mahalanobis"]["err.mean"],
        firefox_unisolated_resist_140ms_NJ_out_to.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_160ms_NJ_out_to.loc["Mahalanobis"]["err.mean"],
        firefox_unisolated_resist_180ms_NJ_out_to.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_200ms_NJ_out_to.loc["Mahalanobis"]["err.mean"],
    ],
    "Mahalanobis(std)": [
        original_out.loc["Mahalanobis"]["err.std"],
        chromium_isolated_to.loc["Mahalanobis"]["err.std"], chromium_unisolated_to.loc["Mahalanobis"]["err.std"],
        chrome_isolated_to.loc["Mahalanobis"]["err.std"], chrome_unisolated_to.loc["Mahalanobis"]["err.std"],
        edge_isolated_to.loc["Mahalanobis"]["err.std"], edge_unisolated_to.loc["Mahalanobis"]["err.std"],
        firefox_isolated_to.loc["Mahalanobis"]["err.std"], firefox_unisolated_to.loc["Mahalanobis"]["err.std"],
        safari_isolated_to.loc["Mahalanobis"]["err.std"], safari_unisolated_to.loc["Mahalanobis"]["err.std"],
        tor_unisolated_to.loc["Mahalanobis"]["err.std"],

        firefox_unisolated_resist_out_to.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_20ms_out_to.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_33ms_out_to.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_40ms_out_to.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_60ms_out_to.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_80ms_out_to.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_100ms_out_to.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_120ms_out_to.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_140ms_out_to.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_160ms_out_to.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_180ms_out_to.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_200ms_out_to.loc["Mahalanobis"]["err.std"],

        firefox_unisolated_resist_NJ_out_to.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_20ms_NJ_out_to.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_33ms_NJ_out_to.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_40ms_NJ_out_to.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_60ms_NJ_out_to.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_80ms_NJ_out_to.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_100ms_NJ_out_to.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_120ms_NJ_out_to.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_140ms_NJ_out_to.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_160ms_NJ_out_to.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_180ms_NJ_out_to.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_200ms_NJ_out_to.loc["Mahalanobis"]["err.std"],
    ],
}

df = pd.DataFrame(df_to, copy=False)
df.to_csv("./password_browser_original.csv", index=False)

In [ ]:
# write data | ot

df_ot = {
    "browser": [
        "Original",
        "Chromium Isolated", "Chromium Unisolated",
        "Chrome Isolated", "Chrome Unisolated",
        "Edge Isolated", "Edge Unisolated",
        "Firefox Isolated", "Firefox Unisolated",
        "Safari Isolated", "Safari Unisolated",
        "Tor Isolated",

        "Firefox RF", "Firefox RF 20ms",
        "Firefox RF 33ms", "Firefox RF 40ms",
        "Firefox RF 60ms", "Firefox RF 80ms",
        "Firefox RF 100ms", "Firefox RF 120ms",
        "Firefox RF 140ms", "Firefox RF 160ms",
        "Firefox RF 180ms", "Firefox RF 200ms",

        "Firefox RF NJ", "Firefox RF NJ 20ms",
        "Firefox RF NJ 33ms", "Firefox RF NJ 40ms",
        "Firefox RF NJ 60ms", "Firefox RF NJ 80ms",
        "Firefox RF NJ 100ms", "Firefox RF NJ 120ms",
        "Firefox RF NJ 140ms", "Firefox RF NJ 160ms",
        "Firefox RF NJ 180ms", "Firefox RF NJ 200ms",
    ],
    "Euclidean(mean)": [
        original_out.loc["Euclidean"]["err.mean"],
        chromium_isolated_ot.loc["Euclidean"]["err.mean"], chromium_unisolated_ot.loc["Euclidean"]["err.mean"],
        chrome_isolated_ot.loc["Euclidean"]["err.mean"], chrome_unisolated_ot.loc["Euclidean"]["err.mean"],
        edge_isolated_ot.loc["Euclidean"]["err.mean"], edge_unisolated_ot.loc["Euclidean"]["err.mean"],
        firefox_isolated_ot.loc["Euclidean"]["err.mean"], firefox_unisolated_ot.loc["Euclidean"]["err.mean"],
        safari_isolated_ot.loc["Euclidean"]["err.mean"], safari_unisolated_ot.loc["Euclidean"]["err.mean"],
        tor_unisolated_ot.loc["Euclidean"]["err.mean"],

        firefox_unisolated_resist_out_ot.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_20ms_out_ot.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_33ms_out_ot.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_40ms_out_ot.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_60ms_out_ot.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_80ms_out_ot.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_100ms_out_ot.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_120ms_out_ot.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_140ms_out_ot.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_160ms_out_ot.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_180ms_out_ot.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_200ms_out_ot.loc["Euclidean"]["err.mean"],

        firefox_unisolated_resist_NJ_out_ot.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_20ms_NJ_out_ot.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_33ms_NJ_out_ot.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_40ms_NJ_out_ot.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_60ms_NJ_out_ot.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_80ms_NJ_out_ot.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_100ms_NJ_out_ot.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_120ms_NJ_out_ot.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_140ms_NJ_out_ot.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_160ms_NJ_out_ot.loc["Euclidean"]["err.mean"],
        firefox_unisolated_resist_180ms_NJ_out_ot.loc["Euclidean"]["err.mean"], firefox_unisolated_resist_200ms_NJ_out_ot.loc["Euclidean"]["err.mean"],
    ],
    "Euclidean(std)": [
        original_out.loc["Euclidean"]["err.std"],
        chromium_isolated_ot.loc["Euclidean"]["err.std"], chromium_unisolated_ot.loc["Euclidean"]["err.std"],
        chrome_isolated_ot.loc["Euclidean"]["err.std"], chrome_unisolated_ot.loc["Euclidean"]["err.std"],
        edge_isolated_ot.loc["Euclidean"]["err.std"], edge_unisolated_ot.loc["Euclidean"]["err.std"],
        firefox_isolated_ot.loc["Euclidean"]["err.std"], firefox_unisolated_ot.loc["Euclidean"]["err.std"],
        safari_isolated_ot.loc["Euclidean"]["err.std"], safari_unisolated_ot.loc["Euclidean"]["err.std"],
        tor_unisolated_ot.loc["Euclidean"]["err.std"],

        firefox_unisolated_resist_out_ot.loc["Euclidean"]["err.std"], firefox_unisolated_resist_20ms_out_ot.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_33ms_out_ot.loc["Euclidean"]["err.std"], firefox_unisolated_resist_40ms_out_ot.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_60ms_out_ot.loc["Euclidean"]["err.std"], firefox_unisolated_resist_80ms_out_ot.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_100ms_out_ot.loc["Euclidean"]["err.std"], firefox_unisolated_resist_120ms_out_ot.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_140ms_out_ot.loc["Euclidean"]["err.std"], firefox_unisolated_resist_160ms_out_ot.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_180ms_out_ot.loc["Euclidean"]["err.std"], firefox_unisolated_resist_200ms_out_ot.loc["Euclidean"]["err.std"],

        firefox_unisolated_resist_NJ_out_ot.loc["Euclidean"]["err.std"], firefox_unisolated_resist_20ms_NJ_out_ot.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_33ms_NJ_out_ot.loc["Euclidean"]["err.std"], firefox_unisolated_resist_40ms_NJ_out_ot.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_60ms_NJ_out_ot.loc["Euclidean"]["err.std"], firefox_unisolated_resist_80ms_NJ_out_ot.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_100ms_NJ_out_ot.loc["Euclidean"]["err.std"], firefox_unisolated_resist_120ms_NJ_out_ot.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_140ms_NJ_out_ot.loc["Euclidean"]["err.std"], firefox_unisolated_resist_160ms_NJ_out_ot.loc["Euclidean"]["err.std"],
        firefox_unisolated_resist_180ms_NJ_out_ot.loc["Euclidean"]["err.std"], firefox_unisolated_resist_200ms_NJ_out_ot.loc["Euclidean"]["err.std"],
    ],
    
    "Manhattan(mean)": [
        original_out.loc["Manhattan"]["err.mean"],
        chromium_isolated_ot.loc["Manhattan"]["err.mean"], chromium_unisolated_ot.loc["Manhattan"]["err.mean"],
        chrome_isolated_ot.loc["Manhattan"]["err.mean"], chrome_unisolated_ot.loc["Manhattan"]["err.mean"],
        edge_isolated_ot.loc["Manhattan"]["err.mean"], edge_unisolated_ot.loc["Manhattan"]["err.mean"],
        firefox_isolated_ot.loc["Manhattan"]["err.mean"], firefox_unisolated_ot.loc["Manhattan"]["err.mean"],
        safari_isolated_ot.loc["Manhattan"]["err.mean"], safari_unisolated_ot.loc["Manhattan"]["err.mean"],
        tor_unisolated_ot.loc["Manhattan"]["err.mean"],

        firefox_unisolated_resist_out_ot.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_20ms_out_ot.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_33ms_out_ot.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_40ms_out_ot.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_60ms_out_ot.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_80ms_out_ot.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_100ms_out_ot.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_120ms_out_ot.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_140ms_out_ot.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_160ms_out_ot.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_180ms_out_ot.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_200ms_out_ot.loc["Manhattan"]["err.mean"],

        firefox_unisolated_resist_NJ_out_ot.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_20ms_NJ_out_ot.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_33ms_NJ_out_ot.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_40ms_NJ_out_ot.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_60ms_NJ_out_ot.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_80ms_NJ_out_ot.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_100ms_NJ_out_ot.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_120ms_NJ_out_ot.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_140ms_NJ_out_ot.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_160ms_NJ_out_ot.loc["Manhattan"]["err.mean"],
        firefox_unisolated_resist_180ms_NJ_out_ot.loc["Manhattan"]["err.mean"], firefox_unisolated_resist_200ms_NJ_out_ot.loc["Manhattan"]["err.mean"],
    ],
    "Manhattan(std)": [
        original_out.loc["Manhattan"]["err.std"],
        chromium_isolated_ot.loc["Manhattan"]["err.std"], chromium_unisolated_ot.loc["Manhattan"]["err.std"],
        chrome_isolated_ot.loc["Manhattan"]["err.std"], chrome_unisolated_ot.loc["Manhattan"]["err.std"],
        edge_isolated_ot.loc["Manhattan"]["err.std"], edge_unisolated_ot.loc["Manhattan"]["err.std"],
        firefox_isolated_ot.loc["Manhattan"]["err.std"], firefox_unisolated_ot.loc["Manhattan"]["err.std"],
        safari_isolated_ot.loc["Manhattan"]["err.std"], safari_unisolated_ot.loc["Manhattan"]["err.std"],
        tor_unisolated_ot.loc["Manhattan"]["err.std"],

        firefox_unisolated_resist_out_ot.loc["Manhattan"]["err.std"], firefox_unisolated_resist_20ms_out_ot.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_33ms_out_ot.loc["Manhattan"]["err.std"], firefox_unisolated_resist_40ms_out_ot.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_60ms_out_ot.loc["Manhattan"]["err.std"], firefox_unisolated_resist_80ms_out_ot.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_100ms_out_ot.loc["Manhattan"]["err.std"], firefox_unisolated_resist_120ms_out_ot.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_140ms_out_ot.loc["Manhattan"]["err.std"], firefox_unisolated_resist_160ms_out_ot.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_180ms_out_ot.loc["Manhattan"]["err.std"], firefox_unisolated_resist_200ms_out_ot.loc["Manhattan"]["err.std"],

        firefox_unisolated_resist_NJ_out_ot.loc["Manhattan"]["err.std"], firefox_unisolated_resist_20ms_NJ_out_ot.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_33ms_NJ_out_ot.loc["Manhattan"]["err.std"], firefox_unisolated_resist_40ms_NJ_out_ot.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_60ms_NJ_out_ot.loc["Manhattan"]["err.std"], firefox_unisolated_resist_80ms_NJ_out_ot.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_100ms_NJ_out_ot.loc["Manhattan"]["err.std"], firefox_unisolated_resist_120ms_NJ_out_ot.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_140ms_NJ_out_ot.loc["Manhattan"]["err.std"], firefox_unisolated_resist_160ms_NJ_out_ot.loc["Manhattan"]["err.std"],
        firefox_unisolated_resist_180ms_NJ_out_ot.loc["Manhattan"]["err.std"], firefox_unisolated_resist_200ms_NJ_out_ot.loc["Manhattan"]["err.std"],
    ],
    
    "ManhattanScaled(mean)": [
        original_out.loc["ManhattanScaled"]["err.mean"],
        chromium_isolated_ot.loc["ManhattanScaled"]["err.mean"], chromium_unisolated_ot.loc["ManhattanScaled"]["err.mean"],
        chrome_isolated_ot.loc["ManhattanScaled"]["err.mean"], chrome_unisolated_ot.loc["ManhattanScaled"]["err.mean"],
        edge_isolated_ot.loc["ManhattanScaled"]["err.mean"], edge_unisolated_ot.loc["ManhattanScaled"]["err.mean"],
        firefox_isolated_ot.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_ot.loc["ManhattanScaled"]["err.mean"],
        safari_isolated_ot.loc["ManhattanScaled"]["err.mean"], safari_unisolated_ot.loc["ManhattanScaled"]["err.mean"],
        tor_unisolated_ot.loc["ManhattanScaled"]["err.mean"],

        firefox_unisolated_resist_out_ot.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_20ms_out_ot.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_33ms_out_ot.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_40ms_out_ot.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_60ms_out_ot.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_80ms_out_ot.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_100ms_out_ot.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_120ms_out_ot.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_140ms_out_ot.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_160ms_out_ot.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_180ms_out_ot.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_200ms_out_ot.loc["ManhattanScaled"]["err.mean"],

        firefox_unisolated_resist_NJ_out_ot.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_20ms_NJ_out_ot.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_33ms_NJ_out_ot.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_40ms_NJ_out_ot.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_60ms_NJ_out_ot.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_80ms_NJ_out_ot.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_100ms_NJ_out_ot.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_120ms_NJ_out_ot.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_140ms_NJ_out_ot.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_160ms_NJ_out_ot.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_180ms_NJ_out_ot.loc["ManhattanScaled"]["err.mean"], firefox_unisolated_resist_200ms_NJ_out_ot.loc["ManhattanScaled"]["err.mean"],
    ],
    "ManhattanScaled(std)": [
        original_out.loc["ManhattanScaled"]["err.std"],
        chromium_isolated_ot.loc["ManhattanScaled"]["err.std"], chromium_unisolated_ot.loc["ManhattanScaled"]["err.std"],
        chrome_isolated_ot.loc["ManhattanScaled"]["err.std"], chrome_unisolated_ot.loc["ManhattanScaled"]["err.std"],
        edge_isolated_ot.loc["ManhattanScaled"]["err.std"], edge_unisolated_ot.loc["ManhattanScaled"]["err.std"],
        firefox_isolated_ot.loc["ManhattanScaled"]["err.std"], firefox_unisolated_ot.loc["ManhattanScaled"]["err.std"],
        safari_isolated_ot.loc["ManhattanScaled"]["err.std"], safari_unisolated_ot.loc["ManhattanScaled"]["err.std"],
        tor_unisolated_ot.loc["ManhattanScaled"]["err.std"],

        firefox_unisolated_resist_out_ot.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_20ms_out_ot.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_33ms_out_ot.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_40ms_out_ot.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_60ms_out_ot.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_80ms_out_ot.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_100ms_out_ot.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_120ms_out_ot.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_140ms_out_ot.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_160ms_out_ot.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_180ms_out_ot.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_200ms_out_ot.loc["ManhattanScaled"]["err.std"],

        firefox_unisolated_resist_NJ_out_ot.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_20ms_NJ_out_ot.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_33ms_NJ_out_ot.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_40ms_NJ_out_ot.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_60ms_NJ_out_ot.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_80ms_NJ_out_ot.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_100ms_NJ_out_ot.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_120ms_NJ_out_ot.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_140ms_NJ_out_ot.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_160ms_NJ_out_ot.loc["ManhattanScaled"]["err.std"],
        firefox_unisolated_resist_180ms_NJ_out_ot.loc["ManhattanScaled"]["err.std"], firefox_unisolated_resist_200ms_NJ_out_ot.loc["ManhattanScaled"]["err.std"],
    ],
    
    "Mahalanobis(mean)": [
        original_out.loc["Mahalanobis"]["err.mean"],
        chromium_isolated_ot.loc["Mahalanobis"]["err.mean"], chromium_unisolated_ot.loc["Mahalanobis"]["err.mean"],
        chrome_isolated_ot.loc["Mahalanobis"]["err.mean"], chrome_unisolated_ot.loc["Mahalanobis"]["err.mean"],
        edge_isolated_ot.loc["Mahalanobis"]["err.mean"], edge_unisolated_ot.loc["Mahalanobis"]["err.mean"],
        firefox_isolated_ot.loc["Mahalanobis"]["err.mean"], firefox_unisolated_ot.loc["Mahalanobis"]["err.mean"],
        safari_isolated_ot.loc["Mahalanobis"]["err.mean"], safari_unisolated_ot.loc["Mahalanobis"]["err.mean"],
        tor_unisolated_ot.loc["Mahalanobis"]["err.mean"],

        firefox_unisolated_resist_out_ot.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_20ms_out_ot.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_33ms_out_ot.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_40ms_out_ot.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_60ms_out_ot.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_80ms_out_ot.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_100ms_out_ot.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_120ms_out_ot.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_140ms_out_ot.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_160ms_out_ot.loc["ManhattanScaled"]["err.mean"],
        firefox_unisolated_resist_180ms_out_ot.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_200ms_out_ot.loc["ManhattanScaled"]["err.mean"],

        firefox_unisolated_resist_NJ_out_ot.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_20ms_NJ_out_ot.loc["Mahalanobis"]["err.mean"],
        firefox_unisolated_resist_33ms_NJ_out_ot.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_40ms_NJ_out_ot.loc["Mahalanobis"]["err.mean"],
        firefox_unisolated_resist_60ms_NJ_out_ot.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_80ms_NJ_out_ot.loc["Mahalanobis"]["err.mean"],
        firefox_unisolated_resist_100ms_NJ_out_ot.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_120ms_NJ_out_ot.loc["Mahalanobis"]["err.mean"],
        firefox_unisolated_resist_140ms_NJ_out_ot.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_160ms_NJ_out_ot.loc["Mahalanobis"]["err.mean"],
        firefox_unisolated_resist_180ms_NJ_out_ot.loc["Mahalanobis"]["err.mean"], firefox_unisolated_resist_200ms_NJ_out_ot.loc["Mahalanobis"]["err.mean"],
    ],
    "Mahalanobis(std)": [
        original_out.loc["Mahalanobis"]["err.std"],
        chromium_isolated_ot.loc["Mahalanobis"]["err.std"], chromium_unisolated_ot.loc["Mahalanobis"]["err.std"],
        chrome_isolated_ot.loc["Mahalanobis"]["err.std"], chrome_unisolated_ot.loc["Mahalanobis"]["err.std"],
        edge_isolated_ot.loc["Mahalanobis"]["err.std"], edge_unisolated_ot.loc["Mahalanobis"]["err.std"],
        firefox_isolated_ot.loc["Mahalanobis"]["err.std"], firefox_unisolated_ot.loc["Mahalanobis"]["err.std"],
        safari_isolated_ot.loc["Mahalanobis"]["err.std"], safari_unisolated_ot.loc["Mahalanobis"]["err.std"],
        tor_unisolated_ot.loc["Mahalanobis"]["err.std"],

        firefox_unisolated_resist_out_ot.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_20ms_out_ot.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_33ms_out_ot.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_40ms_out_ot.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_60ms_out_ot.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_80ms_out_ot.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_100ms_out_ot.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_120ms_out_ot.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_140ms_out_ot.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_160ms_out_ot.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_180ms_out_ot.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_200ms_out_ot.loc["Mahalanobis"]["err.std"],

        firefox_unisolated_resist_NJ_out_ot.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_20ms_NJ_out_ot.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_33ms_NJ_out_ot.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_40ms_NJ_out_ot.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_60ms_NJ_out_ot.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_80ms_NJ_out_ot.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_100ms_NJ_out_ot.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_120ms_NJ_out_ot.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_140ms_NJ_out_ot.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_160ms_NJ_out_ot.loc["Mahalanobis"]["err.std"],
        firefox_unisolated_resist_180ms_NJ_out_ot.loc["Mahalanobis"]["err.std"], firefox_unisolated_resist_200ms_NJ_out_ot.loc["Mahalanobis"]["err.std"],
    ],
}

df = pd.DataFrame(df_ot, copy=False)
df.to_csv("./password_original_browser.csv", index=False)

In [ ]:
# plot data
# plot authentification
x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
labels = [
    'Original', '16.66ms\n(Default)', 
    '20ms', 
    #'33.33ms', 
    '40ms', '60ms', 
    '80ms', '100ms', 
    '120ms',  '140ms', 
    '160ms', '180ms',
    '200ms'
]
categories = [
    "Enrollment: Original, Test: Modified"
    "Enrollment: Modified, Test: Modified"
    "Enrollment: Modified, Test: Original"
]

y_ticks = np.arange(0,110,10)

# ot

y_euclidean_ot = [
    original_out.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_out_ot.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_20ms_out_ot.iloc[0]['err.mean'] * 100,
    #firefox_unisolated_resist_33ms_out_ot.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_40ms_out_ot.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_60ms_out_ot.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_80ms_out_ot.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_100ms_out_ot.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_120ms_out_ot.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_140ms_out_ot.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_160ms_out_ot.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_180ms_out_ot.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_200ms_out_ot.iloc[0]['err.mean'] * 100,
]

y_manhattan_ot = [
    original_out.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_out_ot.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_20ms_out_ot.iloc[1]['err.mean'] * 100,
    #firefox_unisolated_resist_33ms_out_ot.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_40ms_out_ot.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_60ms_out_ot.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_80ms_out_ot.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_100ms_out_ot.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_120ms_out_ot.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_140ms_out_ot.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_160ms_out_ot.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_180ms_out_ot.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_200ms_out_ot.iloc[1]['err.mean'] * 100,
]

y_manhattan_scaled_ot = [
    original_out.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_out_ot.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_20ms_out_ot.iloc[2]['err.mean'] * 100,
    #firefox_unisolated_resist_33ms_out_ot.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_40ms_out_ot.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_60ms_out_ot.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_80ms_out_ot.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_100ms_out_ot.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_120ms_out_ot.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_140ms_out_ot.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_160ms_out_ot.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_180ms_out_ot.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_200ms_out_ot.iloc[2]['err.mean'] * 100,
]

y_mahalanobis_ot = [
    original_out.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_out_ot.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_20ms_out_ot.iloc[3]['err.mean'] * 100,
    #firefox_unisolated_resist_33ms_out_ot.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_40ms_out_ot.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_60ms_out_ot.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_80ms_out_ot.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_100ms_out_ot.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_120ms_out_ot.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_140ms_out_ot.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_160ms_out_ot.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_180ms_out_ot.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_200ms_out_ot.iloc[3]['err.mean'] * 100,
]

# tt

y_euclidean_tt = [
    original_out.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_out_tt.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_20ms_out_tt.iloc[0]['err.mean'] * 100,
    #firefox_unisolated_resist_33ms_out_tt.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_40ms_out_tt.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_60ms_out_tt.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_80ms_out_tt.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_100ms_out_tt.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_120ms_out_tt.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_140ms_out_tt.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_160ms_out_tt.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_180ms_out_tt.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_200ms_out_tt.iloc[0]['err.mean'] * 100,
]

y_manhattan_tt = [
    original_out.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_out_tt.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_20ms_out_tt.iloc[1]['err.mean'] * 100,
    #firefox_unisolated_resist_33ms_out_tt.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_40ms_out_tt.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_60ms_out_tt.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_80ms_out_tt.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_100ms_out_tt.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_120ms_out_tt.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_140ms_out_tt.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_160ms_out_tt.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_180ms_out_tt.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_200ms_out_tt.iloc[1]['err.mean'] * 100,
]

y_manhattan_scaled_tt = [
    original_out.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_out_tt.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_20ms_out_tt.iloc[2]['err.mean'] * 100,
    #firefox_unisolated_resist_33ms_out_tt.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_40ms_out_tt.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_60ms_out_tt.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_80ms_out_tt.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_100ms_out_tt.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_120ms_out_tt.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_140ms_out_tt.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_160ms_out_tt.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_180ms_out_tt.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_200ms_out_tt.iloc[2]['err.mean'] * 100,
]

y_mahalanobis_tt = [
    original_out.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_out_tt.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_20ms_out_tt.iloc[3]['err.mean'] * 100,
    #firefox_unisolated_resist_33ms_out_tt.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_40ms_out_tt.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_60ms_out_tt.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_80ms_out_tt.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_100ms_out_tt.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_120ms_out_tt.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_140ms_out_tt.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_160ms_out_tt.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_180ms_out_tt.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_200ms_out_tt.iloc[3]['err.mean'] * 100,
]

# to

y_euclidean_to = [
    original_out.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_out_to.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_20ms_out_to.iloc[0]['err.mean'] * 100,
    #firefox_unisolated_resist_33ms_out_to.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_40ms_out_to.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_60ms_out_to.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_80ms_out_to.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_100ms_out_to.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_120ms_out_to.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_140ms_out_to.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_160ms_out_to.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_180ms_out_to.iloc[0]['err.mean'] * 100,
    firefox_unisolated_resist_200ms_out_to.iloc[0]['err.mean'] * 100,
]

y_manhattan_to = [
    original_out.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_out_to.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_20ms_out_to.iloc[1]['err.mean'] * 100,
    #firefox_unisolated_resist_33ms_out_to.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_40ms_out_to.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_60ms_out_to.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_80ms_out_to.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_100ms_out_to.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_120ms_out_to.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_140ms_out_to.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_160ms_out_to.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_180ms_out_to.iloc[1]['err.mean'] * 100,
    firefox_unisolated_resist_200ms_out_to.iloc[1]['err.mean'] * 100,
]

y_manhattan_scaled_to = [
    original_out.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_out_to.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_20ms_out_to.iloc[2]['err.mean'] * 100,
    #firefox_unisolated_resist_33ms_out_to.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_40ms_out_to.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_60ms_out_to.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_80ms_out_to.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_100ms_out_to.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_120ms_out_to.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_140ms_out_to.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_160ms_out_to.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_180ms_out_to.iloc[2]['err.mean'] * 100,
    firefox_unisolated_resist_200ms_out_to.iloc[2]['err.mean'] * 100,
]

y_mahalanobis_to = [
    original_out.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_out_to.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_20ms_out_to.iloc[3]['err.mean'] * 100,
    #firefox_unisolated_resist_33ms_out_to.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_40ms_out_to.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_60ms_out_to.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_80ms_out_to.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_100ms_out_to.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_120ms_out_to.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_140ms_out_to.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_160ms_out_to.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_180ms_out_to.iloc[3]['err.mean'] * 100,
    firefox_unisolated_resist_200ms_out_to.iloc[3]['err.mean'] * 100,
]

In [ ]:
# plot | legend right

font_size = 26
font_size_ticks = font_size- 2

x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
labels = [
    'Orig.', '16.66\n(Default)', 
    '20', 
    #'33.33ms', 
    '40', '60', 
    '80', '100', 
    '120', '140', 
    '160', '180',
    '200'
]
y_ticks = np.arange(0,120,20)

categories = ["Euclidean","Manhattan","Manhattan Scaled","Mahalanobis"]

figure = plt.figure(layout='constrained', figsize=(28, 5))
ax = figure.add_subplot(111)

linewidth = 4.0

p1, = ax.plot(x, y_euclidean_ot,  label='',color="violet", linewidth=linewidth)
p2, = ax.plot(x, y_manhattan_ot,  label='', color="gray", linewidth=linewidth)
p3, = ax.plot(x, y_manhattan_scaled_ot, label='', color="green", linewidth=linewidth)
p4, = ax.plot(x, y_mahalanobis_ot, label='', color="blue", linewidth=linewidth)

p5, = ax.plot(x, y_euclidean_tt,  label='', linestyle=':',color="violet", linewidth=linewidth)
p6, = ax.plot(x, y_manhattan_tt,  label='', linestyle=':', color="gray", linewidth=linewidth)
p7, = ax.plot(x, y_manhattan_scaled_tt, label='', linestyle=':', color="green", linewidth=linewidth)
p8, = ax.plot(x, y_mahalanobis_tt, label='', linestyle=':', color="blue", linewidth=linewidth)

p9, = ax.plot(x, y_euclidean_to,  label='', linestyle='dashed', color="violet", linewidth=linewidth)
p10, = ax.plot(x, y_manhattan_to,  label='', linestyle='dashed', color="gray", linewidth=linewidth)
p11, = ax.plot(x, y_manhattan_scaled_to, label='', linestyle='dashed', color="green", linewidth=linewidth)
p12, = ax.plot(x, y_mahalanobis_to, label='', linestyle='dashed', color="blue", linewidth=linewidth)


# create blank rectangle
extra = Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0)

# create organized list containing all handles for table. Extra represent empty space
legend_handle = [
    extra,extra,extra,extra,extra,
    extra,extra,extra,extra,extra,
    extra,extra,extra,extra,extra, extra,p1, p2, p3, p4, extra, p5, p6, p7, p8, extra, p9, p10, p11, p12]

# define the labels
labels_legend = [
    r"$\bf{Enrollment}$: Original, $\bf{Test}$: Modified", "   Euclidean", "   Manhattan", "   Manhattan Scaled", "   Mahalanobis", 
    r"$\bf{Enrollment}$: Modified, $\bf{Test}$: Modified", "   Euclidean", "   Manhattan", "   Manhattan Scaled", "   Mahalanobis", 
    r"$\bf{Enrollment}$: Modified, $\bf{Test}$: Original", "   Euclidean", "   Manhattan", "   Manhattan Scaled", "   Mahalanobis",
    ]
label_empty = [" ", " ", " ", " ", " "]

# organize labels for table construction
legend_labels = np.concatenate([labels_legend, label_empty * 3])

# create legend
figure.legend(legend_handle, legend_labels, ncol = 2, loc='outside right center', handletextpad = -0.5, fontsize=font_size_ticks, frameon=False)




plt.title("", fontsize=font_size)
plt.xlabel("Timer precision setting for Resist Fingerprinting [ms]", fontsize=font_size)
plt.ylabel("Equal error rate [%]", fontsize=font_size)
plt.xticks(x, labels, rotation=0, fontsize=font_size_ticks)
plt.yticks(y_ticks, fontsize=font_size_ticks)
plt.grid(axis="y", alpha=0.4)
plt.savefig('./password_firefox.pdf', bbox_inches="tight", pad_inches=0.1)
plt.show()

In [ ]:
# plot | legend below

font_size = 20
font_size_ticks = font_size- 2

x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
labels = [
    'Orig.', '16.66\n(Default)', 
    '20', 
    #'33.33ms', 
    '40', '60', 
    '80', '100', 
    '120', '140', 
    '160', '180',
    '200'
]
y_ticks = np.arange(0,120,20)

categories = ["Euclidean","Manhattan","Manhattan Scaled","Mahalanobis"]

figure = plt.figure(layout='constrained', figsize=(18, 5))
ax = figure.add_subplot(111)

linewidth = 4.0

p1, = ax.plot(x, y_euclidean_ot,  label='',color="violet", linewidth=linewidth)
p2, = ax.plot(x, y_manhattan_ot,  label='', color="gray", linewidth=linewidth)
p3, = ax.plot(x, y_manhattan_scaled_ot, label='', color="green", linewidth=linewidth)
p4, = ax.plot(x, y_mahalanobis_ot, label='', color="blue", linewidth=linewidth)

p5, = ax.plot(x, y_euclidean_tt,  label='', linestyle=':',color="violet", linewidth=linewidth)
p6, = ax.plot(x, y_manhattan_tt,  label='', linestyle=':', color="gray", linewidth=linewidth)
p7, = ax.plot(x, y_manhattan_scaled_tt, label='', linestyle=':', color="green", linewidth=linewidth)
p8, = ax.plot(x, y_mahalanobis_tt, label='', linestyle=':', color="blue", linewidth=linewidth)

p9, = ax.plot(x, y_euclidean_to,  label='', linestyle='dashed', color="violet", linewidth=linewidth)
p10, = ax.plot(x, y_manhattan_to,  label='', linestyle='dashed', color="gray", linewidth=linewidth)
p11, = ax.plot(x, y_manhattan_scaled_to, label='', linestyle='dashed', color="green", linewidth=linewidth)
p12, = ax.plot(x, y_mahalanobis_to, label='', linestyle='dashed', color="blue", linewidth=linewidth)


# create blank rectangle
extra = Rectangle((0, 0), 0, 0, fc="w", fill=False, edgecolor='none', linewidth=0)

# create organized list containing all handles for table. Extra represent empty space
legend_handle = [
    extra,p1,p2,p3,p4,
    extra,p5,p6,p7,p8,
    extra,p9,p10,p11,p12]

# define the labels
labels_legend = [
    r"$\bf{Enrollment}$: Original, " + "\n" + r"$\bf{Test}$: Modified", "Euclidean", "Manhattan", "Manhattan Scaled", "Mahalanobis",
    r"$\bf{Enrollment}$: Modified, " + "\n" + r"$\bf{Test}$: Modified", "Euclidean", "Manhattan", "Manhattan Scaled", "Mahalanobis",
    r"$\bf{Enrollment}$: Modified, " + "\n" + r"$\bf{Test}$: Original", "Euclidean", "Manhattan", "Manhattan Scaled", "Mahalanobis",
]

# organize labels for table construction
legend_labels = np.concatenate([labels_legend])

# create legend
figure.legend(legend_handle, legend_labels, ncol = 3, loc='outside lower center', fontsize=font_size, frameon=False, bbox_to_anchor=(0.5, -0.52))




plt.title("", fontsize=font_size)
plt.xlabel("Timer precision setting for Resist Fingerprinting [ms]", fontsize=font_size)
plt.ylabel("Equal error rate [%]", fontsize=font_size)
plt.xticks(x, labels, rotation=0, fontsize=font_size_ticks)
plt.yticks(y_ticks, fontsize=font_size_ticks)
plt.grid(axis="y", alpha=0.4)
plt.savefig('./password_firefox2.pdf', bbox_inches="tight", pad_inches=0.1)
plt.show()